In [88]:
import json
import cv2
import os

In [89]:
def get_images():
    images=os.listdir('./aug_images')
    temp={}
    for i in images:
        temp[i]=cv2.imread(f'./aug_images/{i}').copy()
    return temp

In [90]:
Main_data=open('./new_data.json').read()
Main_data=json.loads(Main_data)
type(Main_data)

dict

In [91]:
filter=['filename','x','y','width','height','0','1']

In [92]:
def get_depth(d):

    o = []
    def v_check(key,val):

        if key in filter or val in filter :
            o.append(str(f'{key}:{val}'))

    def check(d):
        if isinstance(d, dict):

            for k,v in d.items():

                if isinstance(v, (dict, list)):
                    check(v)

                else:
                    v=str(v)
                    if  v!='False':
                        v_check(k,v)
                   

        elif isinstance(d, (list)):
            for v in d:

                if isinstance(v, (dict, list)):

                    check(v)

                else:
                    pass
                    # v_check(v)

    check(d)
    return o

data=get_depth(Main_data)
# data

['filename:1.jpeg',
 'x:325',
 'y:27',
 'width:328',
 'height:480',
 '0:True',
 'filename:2.jpeg',
 'x:3',
 'y:5',
 'width:151',
 'height:156',
 '0:True',
 'x:161',
 'y:73',
 'width:102',
 'height:70',
 '1:True',
 'filename:3.jpeg',
 'x:50',
 'y:6',
 'width:200',
 'height:145',
 '1:True',
 'filename:4.jpeg',
 'x:22',
 'y:342',
 'width:924',
 'height:1430',
 '1:True',
 'filename:5.jpeg',
 'x:46',
 'y:56',
 'width:732',
 'height:763',
 '0:True',
 'filename:6.jpeg',
 'x:62',
 'y:33',
 'width:184',
 'height:236',
 '0:True',
 'x:242',
 'y:101',
 'width:130',
 'height:169',
 '1:True',
 'filename:7.jpeg',
 'x:587',
 'y:173',
 'width:246',
 'height:355',
 '0:True',
 'x:53',
 'y:209',
 'width:547',
 'height:285',
 '1:True',
 'filename:aug_1_0.jpg',
 'x:0',
 'y:0',
 'width:911',
 'height:720',
 '0:True',
 'filename:aug_1_1.jpg',
 'x:0',
 'y:0',
 'width:995',
 'height:720',
 '0:True',
 'filename:aug_1_2.jpg',
 'x:460',
 'y:27',
 'width:820',
 'height:693',
 '0:True',
 'filename:aug_1_3.jpg',
 'x:

# YOLO PARSER

In [93]:
def normalize_for_yolo(xmin,ymin,w,h,w_img,h_img):
    xcenter = (xmin + w/2) / w_img
    ycenter = (ymin + h/2) / h_img
    w = w / w_img
    h = h / h_img
    return xcenter,ycenter,w,h
if not os.path.exists('./main_data/yolo_annotations'):
    os.mkdir('./main_data/yolo_annotations')
images=get_images()
image=''
file=''
points=[]
for i in data:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        file=open(f'./main_data/yolo_annotations/{i.split(":")[1].split(".")[0]}.txt','w')
    else:
        if i.split(':')[0]=='0' or i.split(':')[0]=='1':
            file.write(f"{i.split(':')[0]} ")
            x,y,w,h=[int(x) for x in points]
            # image=cv2.rectangle(image,(x,y),(x+w,y+h),[0,255,0],2)
            # image=cv2.putText(image,i.split(':')[0],(x,y),cv2.FONT_HERSHEY_SIMPLEX,2,[0,255,0],2)
            # cv2.imshow('',image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            x,y,w,h=normalize_for_yolo(x,y,w,h,image.shape[1],image.shape[0])
            file.write(f"{x} {y} {w} {h}\n")
            points=[]
        else:
            points.append(i.split(":")[1])
            print(points)
file.close()
    

['325']
['325', '27']
['325', '27', '328']
['325', '27', '328', '480']
['3']
['3', '5']
['3', '5', '151']
['3', '5', '151', '156']
['161']
['161', '73']
['161', '73', '102']
['161', '73', '102', '70']
['50']
['50', '6']
['50', '6', '200']
['50', '6', '200', '145']
['22']
['22', '342']
['22', '342', '924']
['22', '342', '924', '1430']
['46']
['46', '56']
['46', '56', '732']
['46', '56', '732', '763']
['62']
['62', '33']
['62', '33', '184']
['62', '33', '184', '236']
['242']
['242', '101']
['242', '101', '130']
['242', '101', '130', '169']
['587']
['587', '173']
['587', '173', '246']
['587', '173', '246', '355']
['53']
['53', '209']
['53', '209', '547']
['53', '209', '547', '285']
['0']
['0', '0']
['0', '0', '911']
['0', '0', '911', '720']
['0']
['0', '0']
['0', '0', '995']
['0', '0', '995', '720']
['460']
['460', '27']
['460', '27', '820']
['460', '27', '820', '693']
['0']
['0', '0']
['0', '0', '759']
['0', '0', '759', '672']
['0']
['0', '6']
['0', '6', '869']
['0', '6', '869', '714']
[

# Unet

In [94]:
def create_mask_image(img):
    im=img.copy()
    im[:,:]=0
    return im
    

In [95]:
if not os.path.exists('./main_data/unet_annotations'):
    os.mkdir('./main_data/unet_annotations')
    if not os.path.exists('./main_data/unet_annotations/labels'):
        os.mkdir('./main_data/unet_annotations/labels')
    if not os.path.exists('./main_data/unet_annotations/train'):
        os.mkdir('./main_data/unet_annotations/train')

def mask_for_unet(filename,image,mask_image,points,color):
    x,y,w,h=[int(x) for x in points]
    cv2.rectangle(mask_image,(x,y),(x+w,y+h),color,-1)
    mask_image=cv2.resize(mask_image,(512,512),cv2.INTER_AREA)
    image=cv2.resize(image,(512,512),cv2.INTER_AREA)
    cv2.imwrite(f'./main_data/unet_annotations/labels/{filename}.jpg',mask_image)
    cv2.imwrite(f'./main_data/unet_annotations/train/{filename}.jpg',image)
    # cv2.imshow('',mask_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


In [96]:
images=get_images()
image=''
mask_image=''
points=[]
for i in data:
    # print(i)
    if i.split(':')[0] == 'filename':
        image=images[i.split(':')[1]]
        filename=i.split(':')[1].split('.')[0]
        mask_image=create_mask_image(image)
    else:
        if i.split(":")[0] in ['x','y','width','height']:
            points.append(i.split(":")[1])
        else:
            if i.split(":")[0]=='0':#dog
                c=255
            elif i.split(":")[0]=='1':#cat
                c=254
            # print(filename,c)
            mask_for_unet(filename,image,mask_image,points,[c,c,c])
            points=[]